In [1]:
import cv2
import numpy as np

img = cv2.imread("HeliView.jpg")
img = cv2.resize(img, (0, 0), None, .25, .25)

gaussianBlurKernel = np.array(([[1, 2, 1], [2, 4, 2], [1, 2, 1]]), np.float32)/9
sharpenKernel = np.array(([[0, -1, 0], [-1, 9, -1], [0, -1, 0]]), np.float32)/9
meanBlurKernel = np.ones((3, 3), np.float32)/9

gaussianBlur = cv2.filter2D(src=img, kernel=gaussianBlurKernel, ddepth=-1)
meanBlur = cv2.filter2D(src=img, kernel=meanBlurKernel, ddepth=-1)
sharpen = cv2.filter2D(src=img, kernel=sharpenKernel, ddepth=-1)

horizontalStack = np.concatenate((img, gaussianBlur, meanBlur, sharpen), axis=1)

cv2.imwrite("Output.jpg", horizontalStack)

cv2.imshow("2D Convolution Example", horizontalStack)

cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.5.1) /tmp/pip-install-8lnbzhlo/opencv-python/opencv/modules/imgproc/src/resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [2]:
#x_train /= 255????
import os

# Get the current working directory
print( os.getcwd())
os.chdir("/home/kurowskik/data_wb_iad")

/home/kurowskik/data_wb_iad


In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA

import numpy as np
from imblearn.under_sampling import RandomUnderSampler

import pandas as pd
df_test = pd.read_csv('data/test_split.txt',
                 delimiter = " ",
                  names=["number", "filename", "illness", "data"]
                     )


df_train = pd.read_csv('data/train_split.txt',
                 delimiter = " ",
                names=["number", "filename", "illness", "data"])

dict_test = pd.Series(df_test.illness.values,index=df_test.filename).to_dict() 
dict_train = pd.Series(df_train.illness.values,index=df_train.filename).to_dict() 

rus_train = RandomUnderSampler( random_state = 2137, sampling_strategy = {'COVID-19':1670,'pneumonia':1670,'normal':1670})
X_resampled_train, y_resampled_train = rus_train.fit_resample( np.array([*dict_train.keys()]).reshape(-1,1) , [*dict_train.values()])

rus_test = RandomUnderSampler( random_state = 2137, sampling_strategy = {'COVID-19':100,'pneumonia':100,'normal':100})
X_resampled_test, y_resampled_test = rus_test.fit_resample( np.array([*dict_test.keys()]).reshape(-1,1) , [*dict_test.values()])

train_resampled = dict()

for i in range( len( X_resampled_train)):
    if y_resampled_train[i] == "normal":
        train_resampled[ str( X_resampled_train[i][0])] = 0
    if y_resampled_train[i] == "COVID-19":
        train_resampled[ str( X_resampled_train[i][0])] = 1
    if y_resampled_train[i] == "pneumonia":
        train_resampled[ str( X_resampled_train[i][0])] = 2
    
test_resampled = dict()

for i in range( len( X_resampled_test)):
    if y_resampled_test[i] == "normal":
        test_resampled[ str( X_resampled_test[i][0])] = 0
    if y_resampled_test[i] == "COVID-19":
        test_resampled[ str( X_resampled_test[i][0])] = 1
    if y_resampled_test[i] == "pneumonia":
        test_resampled[ str( X_resampled_test[i][0])] = 2
    


In [4]:
import numpy as np
import os
from PIL import Image
import pandas as pd

In [5]:
img_path = os.path.join("data", "train")

def load_images(names):
    n_images = len(names)
    images = np.zeros((n_images, 224, 224, 3))
    labels = np.empty(n_images, dtype='object')
    for i in range(n_images):
        name = names[i]
        #print(name)
        image = Image.open(os.path.join(img_path, name))
        image = image.convert('RGB').resize((224,224)) #thumbnail
        image = np.asarray(image)
        images[i] = image
        labels[i] = dict_train[name]
    return images, labels

In [6]:
def encode(y):
    y[y=="normal"]=0
    y[y=="COVID-19"]=1
    y[y=="pneumonia"]=2
    return y.astype(int)

print( os.getcwd())

/home/kurowskik/data_wb_iad


In [7]:

X_train, y_train = load_images( list(train_resampled.keys()))
y_train = encode(y_train)

In [9]:
from PIL import Image, ImageFilter

im = Image.open(X_train[0])

im1 = im.filter(ImageFilter.BLUR)
im1.show()

im2 = im.filter(ImageFilter.MinFilter(3))
im2.show()

im3 = im.filter(ImageFilter.MinFilter) # same as MinFilter(3)
im3.show()

AttributeError: 'numpy.ndarray' object has no attribute 'read'

In [13]:
img_path = os.path.join("data", "train")

def load_images_sharpen(names):
    n_images = len(names)
    images = np.zeros((n_images, 224, 224, 3))
    labels = np.empty(n_images, dtype='object')
    for i in range(n_images):
        name = names[i]
        #print(name)
        image = Image.open(os.path.join(img_path, name))
        image = image.convert('RGB').resize((224,224)) #thumbnail
        image = image.filter(ImageFilter.SHARPEN)
        image = np.asarray(image)
        images[i] = image
        labels[i] = dict_train[name]
    return images, labels

In [14]:

X_train_sharp, y_train_sharp = load_images_sharpen( list(train_resampled.keys()))
y_train_sharp = encode(y_train_sharp)

In [15]:
np.save("data/x_train_undersampled_sh.npy", X_train)
np.save("data/y_train_undersampled_sh.npy", y_train)

In [18]:
img_path = os.path.join("data", "train")

def load_images_test_sh(names):
    n_images = len(names)
    images = np.zeros((n_images, 224, 224, 3))
    labels = np.empty(n_images, dtype='object')
    for i in range(n_images):
        name = names[i]
        #print(name)
        image = Image.open("/home/kurowskik/data_wb_iad/data/test/" + name)
        image = image.convert('RGB').resize((224,224)) #thumbnail
        image = image.filter(ImageFilter.SHARPEN)
        image = np.asarray(image)
        images[i] = image
        labels[i] = test_resampled[name]
    return images, labels

In [19]:
X_test, y_test = load_images_test_sh( list(test_resampled.keys()))

In [21]:
y_test = encode(y_test)

In [22]:
np.save("data/x_test_undersampled_sh.npy", X_test)
np.save("data/y_test_undersampled_sh.npy", y_test)